# Fine-tune Llama 3.1 8B with Unsloth

Adapted from source: https://colab.research.google.com/drive/164cg_O7SV7G8kZr_JXqLd6VC7pd86-1Z#scrollTo=aTaDCGTe78bK
and
https://huggingface.co/docs/trl/sft_trainer

In [3]:
import torch
from trl import SFTConfig, SFTTrainer
from datasets import load_dataset
from transformers import TrainingArguments, TextStreamer

In [4]:
dataset = load_dataset("imdb", split="train")

sft_config = SFTConfig(
    dataset_text_field="text",
    max_seq_length=512,
    output_dir="/tmp",
)
trainer = SFTTrainer(
    "meta-llama/Meta-Llama-3.1-8B",
    train_dataset=dataset,
    args=sft_config,
)
trainer.train()

ConnectionError: Couldn't reach 'imdb' on the Hub (ConnectionError)